In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 12.3 MB/s 


In [ ]:
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/collage/segmentation')

Mounted at /content/drive


In [ ]:
rs = 1234
# 기존 deeplab v3 + 에서 사용했던 이미지 사이즈를 그대로 사용
IMAGE_SIZE = 512
# colab pro + 에서 구동할 수 있는 최대의 배치 사이즈 선택
BATCH_SIZE = 8
# human level 만을 binary로 예측하기때문에 1
NUM_CLASSES = 1

# 여러 실험을 적용하기 위한 랜덤시드 고정
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
seed_everything(rs)

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/collage/segmentation/data/rawdata/"

# 50000여개의 이미지 + 마스크쌍을 가지고옴
all_images = sorted(glob(os.path.join(DATA_DIR, "humanparsing/val/Images/*"))+glob(os.path.join(DATA_DIR, "humanparsing/train/Images/*"))+glob(os.path.join(DATA_DIR, "fashion/JPEGImages/*")))
all_masks = sorted(glob(os.path.join(DATA_DIR, "humanparsing/val/Human/*"))+glob(os.path.join(DATA_DIR, "humanparsing/train/Human/*"))+glob(os.path.join(DATA_DIR, "fashion/SegmentationClassAug/*")))

# validation size 5000으로 설정 실험 재현읋 위한 random_state 고정
from sklearn.model_selection import train_test_split
val_size = 5000
train_images, val_images, train_masks, val_masks  = train_test_split(all_images, all_masks, test_size=val_size, random_state=rs)

In [ ]:
import tensorflow_addons as tfa

def read_image(image_path, mask=False):
  '''
  하나의 이미지 경로에 대한 이미지를 불러오기위한 함수
  image_path : str / 불러올 이미지 경로
  mask : bool / 일반이미지의 경우 정규화를 하고 마스크 이미지의 경우 정교화를 하지않음
  '''
  image = tf.io.read_file(image_path)
  image = tf.image.decode_png(image, channels=3)
  image.set_shape([None, None, 3])
  image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
  if not mask:
      image = image / 255
  return image

def load_data(image, mask):
  '''
  image : str / 불러올 이미지 경로
  mask : str / 불러올 마스크 경로
  '''
  image = read_image(image_list)
  mask = read_image(mask_list, mask=True)
  return image, mask

def random_crop(image, mask, size):
  '''
  이미지와 마스크 쌍을 인풋하면 각각 같은 크기와 위치를 크롭하여 리턴해주는 함수

  image : array / 이미지
  mask : array / 마스크
  '''
  stacked_image = tf.stack([image, mask], axis=0)
  cropped_image = tf.image.random_crop(
      stacked_image, size=[2, size, size, 3])
  img,msk = cropped_image[0], cropped_image[1]
  return img,msk

def augment(input_image, input_mask):
  '''
  확률적으로 여러 augmentation을 두 쌍으로 적용하는 함수

  input_image : array / 이미지
  input_mask : array / 마스크
  '''
  # 랜덤 크로핑 후 다시 512x512로 리사이즈
  if tf.random.uniform(()) > 0.3:
      ratio = tf.random.uniform(shape=[],minval=400, maxval=440, dtype=tf.int32)
      input_image, input_mask = random_crop(input_image, input_mask, ratio)
      input_image = tf.image.resize(input_image, (IMAGE_SIZE, IMAGE_SIZE))
      input_mask = tf.image.resize(input_mask, (IMAGE_SIZE, IMAGE_SIZE))
  # 이미지에 대해 밝기 조절
  input_image = tf.image.random_brightness(input_image, 0.2)
  # 색상 대비 조절
  saturation_factor = tf.random.uniform((),0,2)
  input_image = tf.image.adjust_saturation(input_image, saturation_factor)
  # 좌우반전
  if tf.random.uniform(()) > 0.5:
      input_image = tf.image.flip_left_right(input_image)
      input_mask = tf.image.flip_left_right(input_mask)
  # 회전
  rot_factor = tf.cast(tf.random.uniform(shape=[],minval=-6, maxval=6, dtype=tf.int32), tf.float32)
  angle = np.pi/(12*6) *rot_factor
  input_image = tfa.image.rotate(input_image, angle)
  input_mask = tfa.image.rotate(input_mask, angle)

  return input_image, input_mask

def cast_mask(input_image, input_mask):
  '''
  rgb 채널로 구성되어있는 마스킹 정보를 1채널의 정보로 압축하는 함수 인물에 해당하는 픽셀은 1 아닌 픽셀은 0
  이미지는 그대로 리턴

  input_image : array / 이미지
  input_mask : array / 마스크
  '''
  input_mask = tf.math.reduce_sum(input_mask, axis=2)
  input_mask = input_mask[..., tf.newaxis]
  input_mask = tf.cast(input_mask>0, dtype=tf.float32)
  return input_image,input_mask

def data_generator(image_list, mask_list, augument=True):
  '''
  데이터 제네레이터 정의

  image_list : list / 이미지들의 경로가 담긴 리스트
  mask_list : list / 마스크들의 경로가 담긴 리스트
  '''
  dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
  dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
  if augument:
    dataset = dataset.map(augment)
  dataset = dataset.map(cast_mask)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
  return dataset

# train set에만 agument 적용 
train_dataset = data_generator(train_images, train_masks)
print("Train Dataset:", train_dataset)

val_dataset = data_generator(val_images, val_masks, augument=False)
print("Validation Dataset:", val_dataset)

Train Dataset: <BatchDataset element_spec=(TensorSpec(shape=(8, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(8, 512, 512, 1), dtype=tf.float32, name=None))>
Validation Dataset: <BatchDataset element_spec=(TensorSpec(shape=(8, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(8, 512, 512, 1), dtype=tf.float32, name=None))>


![](https://github.com/lattice-ai/DeepLabV3-Plus/raw/master/assets/deeplabv3_plus_diagram.png)
 ASPP(Atrous Spatial Pyramid Pooling)
 - 여러 rate의 Atrous conv을 사용
 - 다양한 크기의 receptive field를 확인하게 활용
 - 파라미터 수는 늘지않는다는 장점 


인코더 디코더 구조를 차용
- ASPP로 추출한 피처맵을 다시 확장하여 픽셀단위의 개체예측을 실시하였다.

In [ ]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):

    '''
    conv -> batchnorm -> relu 의 과정을 하나의 함수로 압축
    '''
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape

    # 피처맵 전체의 정보를 담는 avg pool
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    # 원사이즈로 복원
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)
    # 다양한 rate의 Atrous Conv
    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)
    # concat 이후 1x1 conv
    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [ ]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    # image net의 데이터로 학습한 EfficientNetV2M 모델을 transfer learning
    effiucientnetV2M = keras.applications.EfficientNetV2M(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    # 백본의 레이어는 일단 freeze 여러실험을 실시
    for layer in effiucientnetV2M.layers:
      if '_bn' not in layer.name:
        layer.trainable = False
    # 피처맵추출
    x = effiucientnetV2M.get_layer("block6a_expand_activation").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    # 저수준의 피처맵추출
    input_b = effiucientnetV2M.get_layer("block2e_add").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same", activation='sigmoid')(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)

In [ ]:
# 인물 or 배경의 binary문제이기에 로스 , metric도 binary 사용
loss = keras.losses.BinaryCrossentropy()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss=loss,
    metrics=["accuracy",    
        tf.keras.metrics.BinaryIoU(),],
)

In [ ]:
from keras import backend
from keras.utils import io_utils
from tensorflow.python.platform import tf_logging as logging

# tf.keras.callbacks.ReduceLROnPlateau를 상속받아 loss가 줄지 않은 경우 먼저 백본을 언프리즈하고 lr을 0.001로 조정하도록함 
class unfreeze_layers_reduce_lr(tf.keras.callbacks.ReduceLROnPlateau):

  def __init__(self,
               monitor='val_loss',
               factor=0.1,
               patience=10,
               verbose=0,
               mode='auto',
               min_delta=1e-4,
               cooldown=0,
               min_lr=0,
               freezed=False,
               **kwargs):
      super().__init__(monitor, factor, patience, verbose, mode, min_delta, cooldown, min_lr)
      self.freezed = False

  def on_epoch_end(self, epoch, logs=None):
    logs = logs or {}
    logs['lr'] = backend.get_value(self.model.optimizer.lr)
    current = logs.get(self.monitor)
    if current is None:
      logging.warning('Learning rate reduction is conditioned on metric `%s` '
                      'which is not available. Available metrics are: %s',
                      self.monitor, ','.join(list(logs.keys())))
    else:
      if self.in_cooldown():
        self.cooldown_counter -= 1
        self.wait = 0

      if self.monitor_op(current, self.best):
        self.best = current
        self.wait = 0

      elif not self.in_cooldown():
        self.wait += 1
        if self.wait >= self.patience:
          
          if not self.freezed:
            for layer in self.model.layers:
                      layer.trainable = True
            self.freezed=True
            old_lr = backend.get_value(self.model.optimizer.lr)
            new_lr = old_lr * 0.1
            backend.set_value(self.model.optimizer.lr, new_lr)
            io_utils.print_msg(
                    f'\nEpoch {epoch +1}: '
                    f'Unfreezing backbone layers and reducing learning rate to {new_lr}')
            self.cooldown_counter = self.cooldown
            self.wait = 0

          else:
            old_lr = backend.get_value(self.model.optimizer.lr)
            if old_lr > np.float32(self.min_lr):
              new_lr = old_lr * self.factor
              new_lr = max(new_lr, self.min_lr)
              backend.set_value(self.model.optimizer.lr, new_lr)
              if self.verbose > 0:
                io_utils.print_msg(
                    f'\nEpoch {epoch +1}: '
                    f'ReduceLROnPlateau reducing learning rate to {new_lr}.')
              self.cooldown_counter = self.cooldown
              self.wait = 0

model_path, model_name = './model_weight/test/', 'test_epoch_'
model_path_best = './model_weight/test.h5'
csv_name = './model_weight/test.csv'

# 5 epoch 마다 모델을 저장하도록함 
custom_saver = CustomSaver(model_path, model_name ,frequency=5)
# 위에서 정의한 loss가 줄지않으면 backbone을 unfreeze 하고 lr을 0.01 -> 0.001로 바꾸어 계속 학습을 이어나감 / lr decay도 계속해서 적용
reduce_lr = unfreeze_layers_reduce_lr(monitor='val_loss', factor=0.5, patience=4, verbose=1, mode='min', min_delta=1e-4, cooldown=0, min_lr=0.0001,)
# 가장 좋았던 모델은 따로 저장
save_best = tf.keras.callbacks.ModelCheckpoint(model_path_best, monitor='val_loss', verbose=0, save_best_only=True,)
# 모델의 로그를 기록
write_log = tf.keras.callbacks.CSVLogger(csv_name, separator=',', append=True)

cb_list = [custom_saver, reduce_lr, save_best, write_log]

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset,
                    callbacks=cb_list, epochs=100)